### Filling missing data in time-series with Linear regression

#### This code makes use of linear regression (either multiple or linear) to fill gaps in time series.

#### You can make use of those two additional conditions in order to proceed with your calculations:
#### 1. You can use a filter to correct just some specific points and not all the dataset. However pay attention that your entire dataset will still be used as potential indepent variables (Xs); 
#### 2. You can make use of the distance as a limitant of your analysis, i.e., you can define a maximum number of points (n) to be used as variable independents (X) of you variable to be predicted (y) and set only the n-closest points to be selected;
#### 3. Moreover, you can also define that only Xs with |t-statistics| higher than 2 will be used in the MLR computation to garantee that the sigficance is higher than 0

Developed by: Thiago Victor Medeiros do Nascimento

##### Observations: 

You will need three files: 
1. The file with the time-series to be filled: I recomend you to save your time-series data as a dataframe with each variable (rain gauge, well, etc) as a column, and with the index as datetime.
2. A file with the coordinates x and y of your points. The file must have as indexes the variable names (spelled the same!), and x as (Lon) and y as (Lat);
3. A optional file for the filter that you will be using, i.g., a list with the variables that you are interested on filling. 
##### In this example you have a 2-years daily precipitation data for 50 rain gauges. 

In [38]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy
import datetime
import warnings
from scipy.spatial.distance import  cdist
from sklearn import linear_model

In [39]:
warnings.simplefilter(action='ignore', category=Warning)

In [40]:
# Set and print your working directory
os.chdir(r'C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\MLR')
print("Current Working Directory " , os.getcwd())

Current Working Directory  C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\MLR


In [41]:
# Read the xlsx data with the coordinates (x,y) of your points
pathcoords =r'C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\MLR\raing_gauges_unfilled_coords_example.xlsx'
coords=pd.read_excel(pathcoords)
coords.rename(columns={"CÓDIGO": "Code"}, inplace=True)
coords.set_index('Code', inplace=True)
coords.head()

,Lon,Lat
Code,,
26J/04UG,215040.6150,98919.8390
24J/02F,219249.0000,146172.0000
26I/02F,204698.0000,106919.0000
23I/01C,198404.2900,158137.5540
28H/01G,189316.7726,71143.1852


In [42]:
# Read the xlsx file with your nonfilled data
pathnonfilleddata =r'C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\MLR\raing_gauges_unfilled_example.xlsx'
nonfilleddata=pd.read_excel(pathnonfilleddata)
nonfilleddata.set_index('dates', inplace=True)
nonfilleddata.head()

,26J/04UG,24J/02F,26I/02F,23I/01C,28H/01G,26M/02G,27K/01UG,26I/03UG,28I/01UG,24J/02UG,...,26L/01UG,25N/01UG,23K/01UG,27J/01UG,24H/01UG,26J/01UG,27J/03C,27L/02G,24I/01C,24K/02UG
dates,,,,,,,,,,,,,,,,,,,,,
1969-10-01,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0
1969-10-02,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,2.8,...,0.0,NaN,2.1,NaN,NaN,0.0,NaN,NaN,1.3,0.0
1969-10-03,NaN,NaN,NaN,NaN,14.6,NaN,NaN,10.0,46.5,6.5,...,2.0,NaN,1.2,NaN,NaN,11.3,NaN,NaN,1.4,0.9
1969-10-04,NaN,NaN,NaN,4.0,9.8,NaN,NaN,21.3,12.5,6.0,...,25.7,NaN,9.6,NaN,NaN,17.3,NaN,NaN,6.7,12.4
1969-10-05,NaN,NaN,NaN,0.0,7.7,NaN,NaN,2.0,8.7,1.4,...,0.6,NaN,8.0,NaN,NaN,0.8,NaN,NaN,2.0,0.8


In [43]:
num_postos = nonfilleddata.shape[1] #Number of points
num_dias = nonfilleddata.shape[0] #Total time lenght 
print("The number of points that you are working is: " + str(num_postos))
print("With a total of: " + str(num_dias) + " mearsurements")

The number of points that you are working is: 50
With a total of: 730 mearsurements


In [44]:
# For the case that you are working with a sub-set of your main dataset, you can define a filter
# with the points that will be corrected. This reduces computation time when you are not interested
# on filling all the points:
# If you are not working with a filter, please select 0:

work_w_filter = input(" Are you working with a subset of your data? (Yes or Not). And if yes, provide a filter xlsx: ")

if work_w_filter == "Yes":
    pathfilter =r'C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\MLR\filter_example.xlsx'
    filtaux=pd.read_excel(pathfilter)
    id_filter = filtaux["CÓDIGO"]    
    num_corrigir = len(id_filter)
    nonfilleddata_corrigir = nonfilleddata[id_filter]
    print("The number of points to be corrected is: " + str(num_corrigir))
else:
    num_corrigir = num_postos
    nonfilleddata_corrigir = nonfilleddata

 Are you working with a subset of your data? (Yes or Not). And if yes, provide a filter xlsx: Yes
The number of points to be corrected is: 1


In [45]:
# Calculate the percentage of failures per point:
desc = nonfilleddata.describe()
perc_erros = pd.DataFrame(index = coords.index)
perc_erros["perc_erros"] = (1 - desc.iloc[0,:]/num_dias)*100
perc_erros.head()

,perc_erros
Code,
26J/04UG,100.000000
24J/02F,100.000000
26I/02F,100.000000
23I/01C,4.520548
28H/01G,0.000000


In [46]:
#Create a distance matrix:
#Convert the coords to a numpy matrix
coords_np = coords[["Lon", "Lat"]].to_numpy()
dist_mat = cdist(coords_np, coords_np,metric = 'euclidean')
#Convert again for a dataframe
dist_mat_df=pd.DataFrame(dist_mat)
dist_mat_df.columns=coords.index
dist_mat_df.index=coords.index
dist_mat_df

Code,26J/04UG,24J/02F,26I/02F,23I/01C,28H/01G,26M/02G,27K/01UG,26I/03UG,28I/01UG,24J/02UG,...,26L/01UG,25N/01UG,23K/01UG,27J/01UG,24H/01UG,26J/01UG,27J/03C,27L/02G,24I/01C,24K/02UG
Code,,,,,,,,,,,,,,,,,,,,,
26J/04UG,0.000000,47439.195013,13075.024350,61510.203051,37858.401500,49514.257550,19791.041485,18079.724718,39390.175318,44302.775413,...,32925.709990,64216.055825,69531.071336,17078.762936,53527.923573,6887.587190,9541.879090,37375.119068,52500.322147,41297.301738
24J/02F,47439.195013,0.000000,41863.225031,24034.899990,80779.089419,57987.791474,61043.895570,51154.841420,86606.451163,7340.561423,...,46454.765588,64312.142127,24342.606208,64241.502471,27665.020954,43988.332499,52374.954749,65869.722375,9647.591547,12750.894714
26I/02F,13075.024350,41863.225031,0.000000,51603.789196,38942.150441,58895.835542,32857.722086,10277.885731,46343.833741,37076.709077,...,41856.151238,72645.646355,65666.896934,27866.985479,42206.758269,17046.864169,22583.620193,49778.746228,44803.518485,39248.530291
23I/01C,61510.203051,24034.899990,51603.789196,0.000000,87467.726479,81748.154263,78707.097827,58045.559461,97837.064332,20612.238242,...,69084.513209,88323.059059,35855.236840,78433.247997,13017.664812,60340.399280,68808.927170,87284.865639,14397.456142,36631.636186
28H/01G,37858.401500,80779.089419,38942.150441,87467.726479,0.000000,83238.991026,43461.387246,29967.482790,19700.348241,75667.483628,...,68358.216114,98672.449735,104583.023922,29821.457053,75932.278992,44442.243435,40827.002296,63964.848298,83022.826902,77540.692356
26M/02G,49514.257550,57987.791474,58895.835542,81748.154263,83238.991026,0.000000,40401.452449,67126.064023,75102.678825,61511.649480,...,17044.163589,15498.141883,64682.841308,53638.548331,81509.072161,42981.775531,43017.894460,23976.894005,67549.292882,45238.197869
27K/01UG,19791.041485,61043.895570,32857.722086,78707.097827,43461.387246,40401.452449,0.000000,35960.972058,35142.256414,59691.315349,...,27811.280271,55895.487235,79948.593384,13658.889762,72093.231305,18478.318922,10470.251194,20912.793106,67896.056873,51736.462591
26I/03UG,18079.724718,51154.841420,10277.885731,58045.559461,29967.482790,67126.064023,35960.972058,0.000000,40518.417347,45769.395342,...,50238.571941,81420.534645,75259.621514,27080.668689,47242.130984,24177.472722,26898.738505,55162.359912,53056.098806,49391.084267
28I/01UG,39390.175318,86606.451163,46343.833741,97837.064332,19700.348241,75102.678825,35142.256414,40518.417347,0.000000,82824.570888,...,62827.009873,90495.858998,108906.478987,24108.231862,87632.893512,44294.529388,37540.210697,52864.170115,90820.227347,80567.375287


In [47]:
#Dataframe that will be filled
filleddata = nonfilleddata_corrigir

In [48]:
# The loop will work for each point to be corrected
# At this point you can choose if you take into consideration the distance as a limitant factor, or not
# In addition, you can choose to use or not the t-statistic as also a limitant

Use_distance = input("Do you want to take into consideration the distance between points? (Yes/ No) ")
Use_t = input("Do you want to take into consideration the t-statistic of your MLR, i.e., only points with t-stat >= 2 will be considered? (Yes/ No) ")

Do you want to take into consideration the distance between points? (Yes/ No) No
Do you want to take into consideration the t-statistic of your MLR, i.e., only points with t-stat >= 2 will be considered? (Yes/ No) Yes


In [49]:
#Maximum number of gauges used for correction. This only makes sense if you want to use the distance between the wells
#as a limitant for your MLR, i.e., if you want to use only the n-closest wells for you MLR. 
num_correcao = 100000

In [50]:
for i in range(num_corrigir):
#for i in range(1):
    name = nonfilleddata_corrigir.columns[i] #Point's name
    index = nonfilleddata_corrigir[name].index[nonfilleddata_corrigir[name].apply(np.isnan)] #Indexes of the point with gaps
    
    #This loop will correct each day with gap in the point
    for j in index:
        
        #Pay attention that only points with no failures at the day to be corrected in the point to be corrected can be used for the model creation and regression
        names_0_that_day = nonfilleddata.columns[nonfilleddata.loc[j].apply(np.isfinite)] #Code of the points with no errors at this day
        num_ava_points = len(names_0_that_day)
        
        # If there are no points with zero failures at that day, the filling cannot take place
        if num_ava_points == 0:
            filleddata[name].loc[j] = np.nan
        
        else:
            if Use_distance == "Yes":

                # Depending on the num_correcao that you defined, it is possible that the number of available points to be used for corrrection is smaller than this maximum number, therefore 
                # the code will have to use only the available points
                if num_ava_points >= num_correcao:
                    nclosest = dist_mat_df[names_0_that_day].loc[name].nsmallest(num_correcao)
                else:
                    nclosest = dist_mat_df[names_0_that_day].loc[name].nsmallest(num_ava_points)
        
                # Name of the closest points to be used for correction
                names_closest = nclosest.index
            else:
                names_closest = names_0_that_day             
                
             
            # Matrix within the [X y] format
            datamatrix = nonfilleddata[names_closest].join(nonfilleddata[name])
            # Rows with NaN in either of our matrix cannot be used for regression, therefore we must drop them
            datamatrix.dropna(inplace = True)
            
            # The len of the matrix is calculated
            len_mat = len(datamatrix)
            
            # If the len of the matrix is lower than 0 (that can happen), it is not possible to fill the gap
            if len_mat <= 1:
                filleddata[name].loc[j] = np.nan
            
            # Else, the calculation can proceed
            else:
                # Save the y and X
                y = datamatrix.iloc[:,-1]
                X = datamatrix.iloc[:,:-1]
                # If the y column is formed only with 0s, it is not possible to proceed
                if (y != 0).any(axis=0):
                    
                    # In addition, columns (names-X) that have only 0 as measurements are as well deleted
                    X = X.loc[:, (X != 0).any(axis=0)]
                    names_closest = X.columns #And the names of the ones used for correction are also updated
                    
                    # If the len of the matrix is lower than 0 (that can happen), it is not possible to fill the gap
                    if len(names_closest) < 1:
                        filleddata[name].loc[j] = np.nan
                    else:
                        #%%Multiple linear regression
                        regr = linear_model.LinearRegression()
                        regr.fit(X, y)
                        
                        #Maybe the matrix will remain as singular even without the 0 columns, thus, we have to test, and if this is the case, we can proceed the calculation do not taking into consideration the |t-stats| <=2
                        newX = pd.DataFrame({"Constant":np.ones(len(X))}, index = X.index).join(X)
                        if Use_t != "Yes" or np.linalg.det(np.dot(newX.T,newX)) ==0:
                            # Gaps filling
                            filleddata[name].loc[j] = regr.predict([nonfilleddata[names_closest].loc[j]])
                    
                        else:
                            #t-statistics
                            params = np.append(regr.intercept_,regr.coef_)
                            predictions = regr.predict(X)
                
                
                            newX = pd.DataFrame({"Constant":np.ones(len(X))}, index = X.index).join(X)
                            MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))

                            var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
                            sd_b = np.sqrt(var_b)
                            ts_b = params/ sd_b

                
                            myDF3 = pd.DataFrame()
                            myDF3["Coefficients"],myDF3["t values"]= [params,ts_b]
                
                
                            myDF4 = myDF3[1:] 
                            myDF4 = myDF4.set_index(names_closest)
    
                            filt = ((myDF4['t values'].abs() >= 2))
                            new_names = myDF4.loc[filt].index
                
                            # We tested if there are points with abs(t-stats) < 2, and if yes, we re-calculate MLR:
                            if len(new_names) == len(names_closest):
                                # Gaps filling
                                filleddata[name].loc[j] = regr.predict([nonfilleddata[names_closest].loc[j]])
                
                            else:
                                # Matrix within the [X y] format
                                datamatrix_new = nonfilleddata[new_names].join(nonfilleddata[name])
                                # Rows with NaN in either of our matrix cannot be used for regression, therefore we must drop them
                                datamatrix_new.dropna(inplace = True)
                 
                                # Save the y and X
                                y_new = datamatrix_new.iloc[:,-1]
                                X_new = datamatrix_new.iloc[:,:-1]
                                # If the y column is formed only with 0s, it is not possible to proceed
                                if (y_new != 0).any(axis=0):
                 
                                    # In addition, columns (names-X) that have only 0 as measurements are as well deleted
                                    X_new = X_new.loc[:, (X_new != 0).any(axis=0)]
                                    new_names = X_new.columns #And the names of the ones used for correction are also updated
                                    # If the len of the matrix is lower than 0 (that can happen), it is not possible to fill the gap
                                    if len(new_names) < 1:
                                        filleddata[name].loc[j] = np.nan
                                    else:
                                        #%%Multiple linear regression
                                        regr_new = linear_model.LinearRegression()
                                        regr_new.fit(X_new, y_new)
             
                                        # Gaps filling
                                        filleddata[name].loc[j] = regr_new.predict([nonfilleddata[new_names].loc[j]])   
                                else:
                                    filleddata[name].loc[j] = np.nan 
                else:
                    filleddata[name].loc[j] = np.nan

In [51]:
# It is possible that negative values will be calculated, thus we replace them per 0:
filleddata[filleddata < 0.1] = 0

In [52]:
filleddata

,23I/01C
dates,
1969-10-01,0.000000
1969-10-02,1.977921
1969-10-03,0.000000
1969-10-04,4.000000
1969-10-05,0.000000
...,...
1971-09-26,0.000000
1971-09-27,0.000000
1971-09-28,0.000000


In [54]:
path_output=r'C:\Users\User\OneDrive\ERASMUS\4_Thesis\python\MLR\results'
filleddata.to_excel(path_output + '/'+'filleddata'+'.xlsx')  

###### Code developed in 2022 March 21th